```{contents}
```
## Compliance

**Compliance** in LangGraph refers to the **set of architectural, operational, and governance mechanisms** that ensure LLM workflows adhere to **legal, regulatory, security, safety, and organizational requirements** while remaining observable, auditable, and controllable in production.

It is not a single feature — it is a **system-wide design discipline** built into how graphs are defined, executed, stored, and governed.

---

### **1. Why Compliance Is Required for LLM Systems**

LLM workflows influence:

* decisions,
* user data,
* finances,
* legal outcomes,
* safety-critical operations.

Therefore they must satisfy:

| Domain     | Requirements                     |
| ---------- | -------------------------------- |
| Legal      | GDPR, HIPAA, SOC2, ISO 27001     |
| Security   | Data protection, access control  |
| Operations | Reliability, traceability        |
| Governance | Model accountability, approvals  |
| Ethics     | Safety, fairness, explainability |

LangGraph provides the **control surfaces** needed to implement these requirements.

---

### **2. Compliance Architecture in LangGraph**

```
User → API → LangGraph Runtime
              |
     ┌────────┴────────┐
 State Store      Execution Engine
     |                    |
 Checkpoints          Policy Enforcement
     |                    |
 Audit Logs         Human Approval Gates
     |                    |
 Compliance DB ← Observability Layer
```

---

### **3. Core Compliance Concepts**

| Layer               | LangGraph Mechanisms                          |
| ------------------- | --------------------------------------------- |
| **Data Governance** | Encrypted state, PII handling, scoped memory  |
| **Access Control**  | Role-based execution, graph-level permissions |
| **Traceability**    | Full execution trace, event logs              |
| **Auditability**    | Immutable checkpoints, signed records         |
| **Human Oversight** | Approval nodes, intervention points           |
| **Safety**          | Policy nodes, validation gates                |
| **Reliability**     | Retries, rollback, fault isolation            |

---

### **4. Compliance Workflow Example**

**Use case:** Financial document approval

```
Ingest → Validate → LLM Analysis → Risk Check → Human Approval → Finalize
```

Each step enforces compliance rules.

---

### **5. Implementing Compliance in a Graph**

#### **State Schema**

```python
class State(TypedDict):
    document: str
    risk_score: float
    approved: bool
    audit_log: list
```

---

#### **Policy Node**

```python
def policy_check(state):
    if state["risk_score"] > 0.7:
        return {"approved": False}
    return {"approved": True}
```

---

#### **Human Approval Gate**

```python
builder.add_node("human_review", human_node)
builder.add_edge("policy", "human_review")
```

---

#### **Audit Logging**

```python
def audit(state, event):
    state["audit_log"].append(event)
```

---

### **6. Production Compliance Controls**

| Control           | How LangGraph Supports It    |
| ----------------- | ---------------------------- |
| Access control    | Role-based routing           |
| PII protection    | Encrypted state store        |
| Explainability    | Step-by-step execution trace |
| Change management | Versioned graphs             |
| Accountability    | Signed execution logs        |
| Failure handling  | Rollback via checkpoints     |

---

### **7. Regulatory Mapping**

| Regulation     | LangGraph Feature                    |
| -------------- | ------------------------------------ |
| GDPR           | Data isolation, deletion, encryption |
| HIPAA          | Access control, audit logs           |
| SOC2           | Observability, monitoring            |
| ISO 27001      | Security + process controls          |
| Financial Regs | Human approval + audit               |

---

### **8. Common Compliance Patterns**

| Pattern              | Purpose                |
| -------------------- | ---------------------- |
| Approval Gate        | Prevent unsafe actions |
| Two-Person Rule      | Dual human approval    |
| Separation of Duties | Split responsibilities |
| Policy-as-Code       | Automated enforcement  |
| Immutable Ledger     | Tamper-proof records   |

---

### **9. Why LangGraph Excels at Compliance**

Traditional LLM pipelines lack:

* persistent state,
* auditability,
* controlled execution,
* enforceable policies.

LangGraph introduces:

> **Stateful governance + deterministic control + human oversight**

making it suitable for **regulated enterprise environments**.

---

### **10. Mental Model**

Think of LangGraph compliance as:

> **LLM reasoning inside a regulated execution engine**

where **every decision is controlled, visible, and accountable**.



### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# ------------------ State Schema ------------------

class State(TypedDict):
    document: str
    risk_score: float
    approved: bool
    audit_log: List[str]

# ------------------ Nodes ------------------

def ingest(state):
    state["audit_log"].append("Document ingested")
    return state

def analyze(state):
    state["risk_score"] = 0.8 if "transfer" in state["document"].lower() else 0.2
    state["audit_log"].append(f"Risk analyzed: {state['risk_score']}")
    return state

def policy(state):
    state["approved"] = state["risk_score"] <= 0.7
    state["audit_log"].append("Policy evaluated")
    return state

def human_review(state):
    if not state["approved"]:
        state["approved"] = True  # Simulate human override
        state["audit_log"].append("Human approved document")
    return state

def finalize(state):
    state["audit_log"].append("Workflow finalized")
    return state

# ------------------ Graph ------------------

builder = StateGraph(State)

builder.add_node("ingest", ingest)
builder.add_node("analyze", analyze)
builder.add_node("policy", policy)
builder.add_node("human", human_review)
builder.add_node("final", finalize)

builder.set_entry_point("ingest")
builder.add_edge("ingest", "analyze")
builder.add_edge("analyze", "policy")

builder.add_conditional_edges("policy", lambda s: "human" if not s["approved"] else "final", {
    "human": "human",
    "final": "final"
})

builder.add_edge("human", "final")
builder.add_edge("final", END)

graph = builder.compile()

# ------------------ Run ------------------

result = graph.invoke({
    "document": "Approve large transfer",
    "risk_score": 0.0,
    "approved": False,
    "audit_log": []
})

print(result)


{'document': 'Approve large transfer', 'risk_score': 0.8, 'approved': True, 'audit_log': ['Document ingested', 'Risk analyzed: 0.8', 'Policy evaluated', 'Human approved document', 'Workflow finalized']}
